In [31]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np

# Parameters
stock = "TSLA"
rolling_period = 252
percentile_value_high = 90
percentile_value_low = 10
start_date = "2021-12-01"
end_date = "2024-12-01"

# Data Acquisition
stock_data = yf.download(stock, start=start_date, end=end_date)
stock_data.reset_index(inplace=True)
stock_data.columns = stock_data.columns.droplevel(1)
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data

[*********************100%***********************]  1 of 1 completed


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-12-01,365.000000,365.000000,390.946655,363.586670,386.899994,68450400
1,2021-12-02,361.533325,361.533325,371.000000,352.216675,366.353333,73114800
2,2021-12-03,338.323334,338.323334,363.526672,333.403320,361.596680,92322000
3,2021-12-06,336.336670,336.336670,340.546661,316.833344,333.836670,81663000
4,2021-12-07,350.583344,350.583344,352.556671,342.269989,348.066681,56084700
...,...,...,...,...,...,...,...
749,2024-11-22,352.559998,352.559998,361.529999,337.700012,341.089996,89140700
750,2024-11-25,338.589996,338.589996,361.929993,338.200012,360.140015,95890900
751,2024-11-26,338.230011,338.230011,346.959991,335.660004,341.000000,62295900
752,2024-11-27,332.890015,332.890015,342.549988,326.589996,341.799988,57896400


Then I will uses the backtrader

In [32]:
# Custom Indicator: Percentile
class Percentile(bt.Indicator):
    lines = ('percentile',)
    params = (('period', rolling_period), ('percentile', percentile_value_high))

    def __init__(self):
        super().__init__()
        self.addminperiod(self.params.period)

    def next(self):
        window = self.data.get(size=self.params.period)
        if len(window) >= self.params.period:
            self.lines.percentile[0] = np.percentile(window, self.params.percentile)

# Strategy Definition
class MeanRevertingStrategy(bt.Strategy):
    params = (
        ('ma_period', rolling_period),
        ('high_percentile', percentile_value_high),
        ('low_percentile', percentile_value_low),
    )

    def __init__(self):
        # Indicators
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.ma_period)
        self.bias = self.data.close / self.sma
        self.high_bias = Percentile(self.bias, period=self.params.ma_period, percentile=self.params.high_percentile)
        self.low_bias = Percentile(self.bias, period=self.params.ma_period, percentile=self.params.low_percentile)

        self.order = None  # To keep track of pending orders
        self.trades = []  # To store trade details

    def log(self, txt, dt=None):
        """ Logging function for this strategy """
        dt = dt or self.data.datetime.date(0)
        print(f"{dt}, {txt}")

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Order is submitted or accepted by broker - Do nothing
            return

        # Order completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f"BUY EXECUTED, Price: {order.executed.price}, Size: {order.executed.size}, Value: {order.executed.value}, Commission: {order.executed.comm}")
            elif order.issell():
                self.log(f"SELL EXECUTED, Price: {order.executed.price}, Size: {order.executed.size}, Value: {order.executed.value}, Commission: {order.executed.comm}")

            # Store trade information
            self.trades.append({
                'Date': self.data.datetime.date(0),
                'Type': 'Buy' if order.isbuy() else 'Sell',
                'Price': order.executed.price,
                'Size': order.executed.size,
                'Value': order.executed.value,
                'Commission': order.executed.comm,
            })

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")

        # Reset order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f"TRADE PROFIT, Gross: {trade.pnl}, Net: {trade.pnlcomm}")

    def next(self):
        cash = self.broker.getcash()
        price = self.data.close[0]
        size = int((cash * 0.10) / price)
        # Entry logic
        if not self.position:
            if self.data.close[0] > self.sma[0]:
                self.order = self.buy(size=size)  # Enter long position

        # Exit logic
        elif self.position:
            if self.data.close[0] < self.sma[0]:  # Price below SMA
                self.order = self.sell(size=self.position.size)
            elif self.bias[0] > self.high_bias[0]:  # Bias exceeds high percentile
                self.order = self.sell(size=self.position.size)
            elif self.data.close[0] < self.data.close[-1] and self.data.close[-1] > self.sma[-1]:  # Reversal
                self.order = self.sell(size=self.position.size)

# Backtesting Environment
if __name__ == "__main__":
    cerebro = bt.Cerebro()

    # Add data to Cerebro
    class PandasData(bt.feeds.PandasData):
        params = (
            ('datetime', 'Date'),
            ('open', 'Open'),
            ('high', 'High'),
            ('low', 'Low'),
            ('close', 'Close'),
            ('volume', 'Volume'),
            ('openinterest', None),
        )

    bt_data = PandasData(dataname=stock_data)
    cerebro.adddata(bt_data)

    # Add strategy
    cerebro.addstrategy(MeanRevertingStrategy)

    # Set initial cash and commission
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.001)

    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analysis")

    # Run the backtest
    print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
    results = cerebro.run()
    print("Ending Portfolio Value: %.2f" % cerebro.broker.getvalue())

    # Extract the strategy instance
    strat = results[0]

    # Print Sharpe Ratio
    sharpe = strat.analyzers.sharpe.get_analysis()
    print(f"\nSharpe Ratio:\n{sharpe}")

    # Print Trade Analysis
    trade_analysis = strat.analyzers.trade_analysis.get_analysis()
    print(f"\nTrade Analysis:\n")
    for key, value in trade_analysis.items():
        print(f"{key}: {value}")

    # Output transactions as a readable DataFrame
    transactions_df = pd.DataFrame(strat.trades)
    print("\nTransaction Log:")
    print(transactions_df)

    # Plot results
    cerebro.plot()


Starting Portfolio Value: 100000.00
2023-12-01, BUY EXECUTED, Price: 233.13999938964844, Size: 41, Value: 9558.739974975586, Commission: 9.558739974975586
2023-12-04, SELL EXECUTED, Price: 235.75, Size: -41, Value: 9558.739974975586, Commission: 9.665750000000001
2023-12-04, TRADE PROFIT, Gross: 107.01002502441406, Net: 87.78553504943848
2023-12-05, BUY EXECUTED, Price: 233.8699951171875, Size: 42, Value: 9822.539794921875, Commission: 9.822539794921875
2023-12-12, SELL EXECUTED, Price: 238.5500030517578, Size: -42, Value: 9822.539794921875, Commission: 10.019100128173829
2023-12-12, TRADE PROFIT, Gross: 196.56033325195312, Net: 176.71869332885743
2023-12-13, BUY EXECUTED, Price: 234.19000244140625, Size: 42, Value: 9835.980102539062, Commission: 9.835980102539063
2023-12-19, SELL EXECUTED, Price: 253.47999572753906, Size: -42, Value: 9835.980102539062, Commission: 10.646159820556642
2023-12-19, TRADE PROFIT, Gross: 810.1797180175781, Net: 789.6975780944824
2023-12-20, BUY EXECUTED, Pr